In [2]:
def first(iterable, default=None):
    return next(iter(iterable), default)  

In [3]:
class KB:

    def __init__(self, sentence=None):
        if sentence:
            self.tell(sentence)

    def tell(self, sentence):
        raise NotImplementedError
    
    def ask(self, query):
        return first(self.ask_generator(query), default=False)
    
    def ask_generator(self, query):
        raise NotImplementedError

    def retract(self, sentence):
        raise NotImplementedError

In [4]:
class Expr:
    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

In [5]:
def conjuncts(s):
    return dissociate('&', [s])

def disjuncts(s):
    if isinstance(s, set):
        return list(s)
    else:
        return [s]

def dissociate(op, args):
    result = []
    return result 

    def collect(subargs):
        for arg in subargs:
            if arg.op == op:
                collect(arg.args)
            else:
                result.append(arg)
            return result 

def unique(seq):
    return list(set(seq))

def remove_all(item, seq):
    if isinstance(seq, str):
        return seq.replace(item, '')
    elif isinstance(seq, set):
        rest = seq.copy()
        rest.remove(item)
        return rest
    else:
        return [x for x in seq if x != item]

def associate(op, args):
    args = dissociate(op, args)

    if len(args) == 0:
        return _op_identity[op]
    else:
      if len(args) == 1:
        return args[0]
      else:
        return Expr(op, *args)

_op_identity = {'&': True, '|': False, '+': 0, '*': 1}

In [6]:
def PLResolution(kb, alpha):
    clauses = [kb] + [alpha]
    new = set()

    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j])
                 for i in range(n) for j in range(i + 1, n)]

        for (ci, cj) in pairs:
            resolvents = pl_resolve(ci, cj)
            if False in resolvents:
                return True
            new = new.union(set(resolvents))
        if new.issubset(set(clauses)):
            return False

        for c in new:
            if c not in clauses:
                clauses.append(c)

def pl_resolve(ci, cj):
    clauses = []
    for di in disjuncts(ci):
        for dj in disjuncts(cj):
          if di != dj or dj == di:
              clauses.append(associate('|', unique(remove_all(di, disjuncts(ci)) + remove_all(dj, disjuncts(cj)))))
        return clauses
    return set(clauses)
  

In [7]:
## Test of PL Resolution Case1 

KB3 = set({'P12'})
print("KB3:" + str(KB3))

alpha = alpha1= {'-P12'}
print("alpha1:" + str(alpha1))

print(PLResolution(KB3,alpha1)) # Correct Answer: True


KB3:{'P12'}
alpha1:{'-P12'}
True


In [8]:
## Test of PL Resolution Case2 

list1 = ['P12', 'B11']
list2 = ['-B11']

output = [list1[i % len(list1)]+list2[i] for i in range(len(list2))]
print("List Sum:" + str(output))

KB2 = set(output)
print("KB2:" + str(KB2))

alpha = alpha1 = {'-P12'}
print("alpha1:" + str(alpha1))

print(PLResolution(KB2, alpha1)) # Correct Answer: True

List Sum:['P12-B11']
KB2:{'P12-B11'}
alpha1:{'-P12'}
True


In [9]:
## Test of PL Resolution Case3 

list1 = ['P12', 'B11']
list2 = ['-B11']

output = [list1[i % len(list1)]+list2[i] for i in range(len(list2))]
print("List Sum:" + str(output))

KB2 = set(output)
print("KB2:" + str(KB2))

alpha = alpha2 = {'P12'}
print("alpha2:" + str(alpha2))

print(PLResolution(KB2, alpha2)) # Correct Answer: False

List Sum:['P12-B11']
KB2:{'P12-B11'}
alpha2:{'P12'}
True


In [10]:
## Test of PL Resolution Case4

listA = ['-P21', 'B11']
listB = ['-B11', 'P12', 'P21']
listC = ['-P12', 'B11']
listD = ['-B11']

res_list = listA + listB + listC + listD
print("List Sum:" + str(res_list))

KB1 = set(res_list)
print("KB1:" + str(KB1))

alpha = alpha2 = {'P12'}
print("alpha2:" + str(alpha2))

print(PLResolution(KB1, alpha2)) # Correct Answer: True

List Sum:['-P21', 'B11', '-B11', 'P12', 'P21', '-P12', 'B11', '-B11']
KB1:{'-P21', '-P12', '-B11', 'P12', 'B11', 'P21'}
alpha2:{'P12'}
True


In [11]:
## Test of PL Resolution Case5

list1 = ['P12', 'B11']
list2 = ['-B11']

output = [list1[i % len(list1)]+list2[i] for i in range(len(list2))]
print("List Sum:" + str(output))

KB2 = set(output)
print("KB2:" + str(KB2))

alpha = alpha3 = {'P12', 'B11'}
print("alpha3:" + str(alpha3))

print(PLResolution(KB2, alpha3)) # Correct Answer: False

List Sum:['P12-B11']
KB2:{'P12-B11'}
alpha3:{'B11', 'P12'}
True


In [12]:
## Test of PL Resolution Case6

listE = ['-P']
listF = ['R']
listG = ['P', 'T']

res_listT = listE + listF + listG 
print("List Sum:" + str(res_listT))

KB4 = set(res_listT)
print("KB4:" + str(KB4))

alpha = alpha4 = {'-T'}
print("alpha4:" + str(alpha4))

print(PLResolution(KB4, alpha4)) # Correct Answer: True

List Sum:['-P', 'R', 'P', 'T']
KB4:{'P', '-P', 'R', 'T'}
alpha4:{'-T'}
True


In [13]:
## Test of PL Resolution Case7

listE = ['-P']
listF = ['R']
listG = ['P', 'T']

res_listT = listE + listF + listG 
print("List Sum:" + str(res_listT))

KB4 = set(res_listT)
print("KB4:" + str(KB4))

alpha = _alpha4 = {'T'}
print("_alpha4:" + str(_alpha4))

print(PLResolution(KB4, _alpha4)) # Correct Answer: False

List Sum:['-P', 'R', 'P', 'T']
KB4:{'P', '-P', 'R', 'T'}
_alpha4:{'T'}
True


In [14]:
## Test of PL Resolution Case8

listH = ['-q', 'p']
listI = ['-p', 'r']
listJ = ['q', 's']
listK = ['-s']

res_listC = listH + listI + listJ + listK
print("List Sum:" + str(res_listC))

KB5 = set(res_listC)
print("KB5:" + str(KB5))

alpha = alpha5 = {'-r'}
print("alpha:" + str(alpha5))

print(PLResolution(KB5, alpha5)) # Correct Answer: True

List Sum:['-q', 'p', '-p', 'r', 'q', 's', '-s']
KB5:{'-p', 's', 'q', 'p', '-s', 'r', '-q'}
alpha:{'-r'}
True


In [15]:
## Test of PL Resolution Case9

listH = ['-q', 'p']
listI = ['-p', 'r']
listJ = ['q', 's']
listK = ['-s']

res_listC = listH + listI + listJ + listK
print("List Sum:" + str(res_listC))

KB5 = set(res_listC)
print("KB5:" + str(KB5))

alpha = _alpha5 = {'r'}
print("_alpha:" + str(_alpha5))

print(PLResolution(KB5, _alpha5)) # Correct Answer: False

List Sum:['-q', 'p', '-p', 'r', 'q', 's', '-s']
KB5:{'-p', 's', 'q', 'p', '-s', 'r', '-q'}
_alpha:{'r'}
True
